# OpenAI Function Calling Basics

This notebook demonstrates basic usage of OpenAI's function calling API. Function calling allows you to define functions that the model can call, making it easier to get structured outputs and perform specific actions.

## Setup

First, let's install and import the required packages.

In [1]:
!pip install openai
from openai import OpenAI
import json

# Initialize the client
client = OpenAI(api_key="sk-prFakePymW-SPbIvnjFAKeaqKmevK5JRlvE4StUoQC0P8A")

## Basic Calculator Example

Let's start with a simple calculator example that can perform basic arithmetic operations.

In [2]:
def calculate(operation: str, x: float, y: float) -> float:
    """Perform basic arithmetic operations"""
    if operation == "add":
        return x + y
    elif operation == "subtract":
        return x - y
    elif operation == "multiply":
        return x * y
    elif operation == "divide":
        if y == 0:
            raise ValueError("Cannot divide by zero")
        return x / y
    else:
        raise ValueError(f"Unknown operation: {operation}")

# Define the function schema
calculator_function = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Perform basic arithmetic operations between two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The arithmetic operation to perform"
                },
                "x": {
                    "type": "number",
                    "description": "The first number"
                },
                "y": {
                    "type": "number",
                    "description": "The second number"
                }
            },
            "required": ["operation", "x", "y"]
        }
    }
}

def process_function_call(user_input: str) -> str:
    """Process user input and call appropriate function"""
    # Get model's response with function call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}],
        tools=[calculator_function],
        tool_choice="auto"
    )

    # Extract function call
    message = response.choices[0].message
    tool_calls = message.tool_calls

    if tool_calls:
        # Process the function call
        function_call = tool_calls[0]
        function_args = json.loads(function_call.function.arguments)

        # Call the function
        if function_call.function.name == "calculate":
            try:
                result = calculate(**function_args)
                return f"The result is: {result}"
            except ValueError as e:
                return f"Error: {str(e)}"

    return "No function was called"

# Test the calculator
test_inputs = [
    "What is 25 plus 15?",
    "Calculate 100 divided by 5",
    "Multiply 12 by 8",
    "What is 50 minus 27?",
    "Divide 10 by 0"  # Error case
]

for input_text in test_inputs:
    print(f"Input: {input_text}")
    result = process_function_call(input_text)
    print(f"Result: {result}")
    print("-"*50)

Input: What is 25 plus 15?
Result: The result is: 40
--------------------------------------------------
Input: Calculate 100 divided by 5
Result: The result is: 20.0
--------------------------------------------------
Input: Multiply 12 by 8
Result: The result is: 96
--------------------------------------------------
Input: What is 50 minus 27?
Result: The result is: 23
--------------------------------------------------
Input: Divide 10 by 0
Result: Error: Cannot divide by zero
--------------------------------------------------


## Weather Information Example

Now let's look at a more complex example that simulates getting weather information.

In [3]:
def get_weather(location: str, unit: str = "celsius") -> str:
    """Simulate getting weather information for a location"""
    # This is a mock implementation
    weather_data = {
        "New York": {"celsius": 20, "fahrenheit": 68, "condition": "sunny"},
        "London": {"celsius": 15, "fahrenheit": 59, "condition": "cloudy"},
        "Tokyo": {"celsius": 25, "fahrenheit": 77, "condition": "rainy"},
        "Sydney": {"celsius": 22, "fahrenheit": 72, "condition": "clear"}
    }

    location = location.title()
    if location not in weather_data:
        raise ValueError(f"No weather data available for {location}")

    data = weather_data[location]
    temp = data[unit.lower()]
    return f"The weather in {location} is {data['condition']} with a temperature of {temp}°{unit[0].upper()}"

# Define the function schema
weather_function = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather information for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city to get weather information for"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use"
                }
            },
            "required": ["location"]
        }
    }
}

def process_weather_query(user_input: str) -> str:
    """Process weather-related queries"""
    # Get model's response with function call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}],
        tools=[weather_function],
        tool_choice="auto"
    )

    # Extract function call
    message = response.choices[0].message
    tool_calls = message.tool_calls

    if tool_calls:
        # Process the function call
        function_call = tool_calls[0]
        function_args = json.loads(function_call.function.arguments)

        # Call the function
        if function_call.function.name == "get_weather":
            try:
                result = get_weather(**function_args)
                return result
            except ValueError as e:
                return f"Error: {str(e)}"

    return "No function was called"

# Test the weather function
test_queries = [
    "What's the weather like in New York?",
    "Tell me the temperature in London in Fahrenheit",
    "How's the weather in Tokyo?",
    "What's the temperature in Sydney in Celsius?",
    "What's the weather like in Paris?"  # Error case
]

for query in test_queries:
    print(f"Query: {query}")
    result = process_weather_query(query)
    print(f"Response: {result}")
    print("-"*50)

Query: What's the weather like in New York?
Response: The weather in New York is sunny with a temperature of 20°C
--------------------------------------------------
Query: Tell me the temperature in London in Fahrenheit
Response: The weather in London is cloudy with a temperature of 59°F
--------------------------------------------------
Query: How's the weather in Tokyo?
Response: The weather in Tokyo is rainy with a temperature of 25°C
--------------------------------------------------
Query: What's the temperature in Sydney in Celsius?
Response: The weather in Sydney is clear with a temperature of 22°C
--------------------------------------------------
Query: What's the weather like in Paris?
Response: Error: No weather data available for Paris
--------------------------------------------------


## Key Points

1. Function Definition:
   - Define Python functions that implement the desired functionality
   - Create JSON schemas that describe the functions to the model

2. Function Calling:
   - Pass functions to the model using the `tools` parameter
   - Use `tool_choice="auto"` to let the model decide when to call functions
   - Extract function calls from the response and process them

3. Error Handling:
   - Validate function arguments
   - Handle potential errors gracefully
   - Provide meaningful error messages

4. Best Practices:
   - Use clear function and parameter names
   - Provide detailed descriptions in the function schema
   - Include proper error handling and validation
   - Test with various inputs including edge cases